In [1]:
import json

import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torch.nn.functional as F
import torch_geometric.utils as utils
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATv2Conv
from torch_geometric.nn import GINConv
from sklearn.model_selection import train_test_split
import torch_geometric as tg
import numpy as np
from scipy.sparse import csr_matrix
from scipy import sparse as sp
import numpy as np
import networkx as nx
import sys

In [2]:
def calculatePosEncodings(edge_index, num_nodes):
    edge_index = edge_index.t().tolist()
    edges = [(src, dst) for src, dst in edge_index]

# Create the adjacency matrix in CSR format -> das wird dann für die encodings benutzt!
    rows, cols = zip(*edges)
    data = np.ones(len(rows))
    A = csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))

    ''' this code computes the in_degrees matrix from the edge list. it can later be adapted to compute the in-degrees matrix from the adjacency matrix (however, then, we should
    do some tests with small sample graphs to ensure everything is correct
    '''
    in_degrees_dict = {node: 0 for node in range(num_nodes)}
    # Calculate the in-degrees for each node
    for edge in edges:
        _, dst = edge
        in_degrees_dict[dst] += 1

    in_degrees = np.array([in_degrees_dict[i] for i in range(len(in_degrees_dict))], dtype=float)
    in_degrees = in_degrees.clip(1)  # Clip to ensure no division by zero
    in_degrees = np.power(in_degrees, -0.5)  # Take the element-wise inverse square root

    # Create the sparse diagonal matrix N
    N = sp.diags(in_degrees, dtype=float)

    L = sp.eye(num_nodes) - N * A * N

    #calc eigvals and eigVecs, equivalent to the original code
    EigVal, EigVec = np.linalg.eig(L.toarray())
    idx = EigVal.argsort() # increasing order
    EigVal, EigVec = EigVal[idx], np.real(EigVec[:,idx])

    #pos_enc_dim = hyperparameter!
    pos_enc_dim = 5
    RESULT_POS_ENCODING = torch.from_numpy(EigVec[:,1:pos_enc_dim+1]).float() 
    return RESULT_POS_ENCODING

def calculateLoss(task_loss, batch, num_nodes, positional_encoding):
    #HYPERPARAMETERS
    device = "cpu"
    pos_enc_dim = 1
    alpha_loss: 1e-3
    lambda_loss: 100  # ist auch 100

    #edge_index im korrekten Format definieren
    edge_index = batch.edge_index.t().tolist()
    edge_index = [(src, dst) for src, dst in edge_index]

    # Loss B: Laplacian Eigenvector Loss --------------------------------------------
    n = num_nodes

    # Laplacian 
    rows, cols = zip(*edge_index)
    data = np.ones(len(rows))
    A = csr_matrix((data, (rows, cols)), shape=(num_nodes, num_nodes))

    ''' this code computes the in_degrees matrix from the edge list. it can later be adapted to compute the in-degrees matrix from the adjacency matrix (however, then, we should
    do some tests with small sample graphs to ensure everything is correct'''

    in_degrees_dict = {node: 0 for node in range(num_nodes)}
    # Calculate the in-degrees for each node
    for edge in edge_index:
        _, dst = edge
        in_degrees_dict[dst] += 1

    in_degrees = np.array([in_degrees_dict[i] for i in range(len(in_degrees_dict))], dtype=float)
    in_degrees = in_degrees.clip(1)  # Clip to ensure no division by zero
    in_degrees = np.power(in_degrees, -0.5)  # Take the element-wise inverse square root

    # Create the sparse diagonal matrix N
    N = sp.diags(in_degrees, dtype=float)
    L = sp.eye(num_nodes) - N * A * N

    p = positional_encoding
    pT = torch.transpose(p, 1, 0)
    loss_b_1 = torch.trace(torch.mm(torch.mm(pT, torch.Tensor(L.todense()).to(device)), p))

    '''  TODO: loss_b_2 
    '''

    loss_b = loss_b_1

    #TODO: parameter tunen!
    loss = task_loss + 1e-3* loss_b
    return loss


def precision(predictions, targets, threshold):
    # Apply a threshold to the predictions
    binary_predictions = (predictions >= threshold).astype(int)
    binary_targets = (targets >= threshold).astype(int)

    # Calculate the true positive (TP) and false positive (FP) counts
    TP = np.sum((binary_predictions == 1) & (binary_targets == 1))
    FP = np.sum((binary_predictions == 1) & (binary_targets == 0))

    print("Negative: ")
    print(np.sum(binary_targets == 1))
    print("Positive: ")
    print(np.sum(binary_targets == 0))
    # Calculate precision
    precision_value = TP / (TP + FP)
    return precision_value

def recall(predictions, targets, threshold):
    # Apply a threshold to the predictions
    binary_predictions = (predictions >= threshold).astype(int)
    binary_targets = (targets >= threshold).astype(int)
    # Calculate the true positive (TP) and false negative (FN) counts
    TP = np.sum((binary_predictions == 1) & (binary_targets == 1))
    FN = np.sum((binary_predictions == 0) & (binary_targets == 1))
    # Calculate recall
    recall_value = TP / (TP + FN)
    return recall_value


In [7]:
business_ids=[]

try:
    with open('yelp_dataset/yelp_academic_dataset_business.json', 'r', encoding='utf-8') as json_file:
        for line in json_file:
            try:
                data = json.loads(line)
                business_ids.append(data["business_id"])
            except json.decoder.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
except FileNotFoundError:
    print("File not found.")
print("done business")

review_ids=[]
review_business_ids=[]
review_user_ids=[]
review_stars=[]

try:
    with open('yelp_dataset/yelp_academic_dataset_review.json', 'r', encoding='utf-8') as json_file:
        for i,line in enumerate(json_file):
            try:
                data = json.loads(line)
                review_ids.append(data["review_id"])
                review_business_ids.append(data["business_id"])
                review_user_ids.append(data["user_id"])
                review_stars.append(data["stars"])
            except json.decoder.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
except FileNotFoundError:
    print("File not found.")
print("done reviews")

user_ids=[]

try:
    with open('yelp_dataset/yelp_academic_dataset_user.json', 'r', encoding='utf-8') as json_file:
        for line in json_file:
            try:
                data = json.loads(line)
                user_ids.append(data["user_id"])
            except json.decoder.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
except FileNotFoundError:
    print("File not found.")
print("done users")

done business
done reviews
done users


In [4]:
print(len(user_ids))
print(len(business_ids))
print(len(review_stars))

max_businesses = 1000
max_users = 40000

business_ids = business_ids[:max_businesses]
user_ids = user_ids[:max_users]

print("---------------------------------------------")
print(len(user_ids))
print(len(business_ids))
print(len(review_stars))

1987897
150346
6990280
---------------------------------------------
40000
20000
6990280


In [5]:
user_to_index = {user_id: index for index, user_id in enumerate(set(user_ids))}
business_to_index = {business_id: index for index, business_id in enumerate(set(business_ids))}

user_index_col = [user_to_index[user_id] for user_id in user_ids]
business_index_col = [business_to_index[business_id] for business_id in business_ids]


skip_indexes = []
current_index = -1

for user_id in review_user_ids:
    try:
        current_index = current_index + 1
        tmp = user_to_index[user_id]
    except KeyError:
        skip_indexes.append(current_index)

current_index = -1
for business_id in review_business_ids:
    try:
        current_index = current_index + 1
        tmp = business_to_index[business_id]
    except KeyError:
        skip_indexes.append(current_index)    

skip_indexes = set(skip_indexes)
print(6990280 - len(skip_indexes))

155429


In [6]:
current_index = -1

review_user_index_col = []
for user_id in review_user_ids:
    try:
        current_index = current_index + 1
        if current_index in skip_indexes:
            pass
        else:
            user_index = user_to_index[user_id]
            review_user_index_col.append(user_index)
    except KeyError:
        print("Huh?")        
        
current_index = -1        
review_business_index_col = []
for business_id in review_business_ids:
    try:
        current_index = current_index + 1
        if current_index in skip_indexes:
            pass
        else:
            business_index = business_to_index[business_id]
            review_business_index_col.append(business_index)
    except KeyError:
        print("Huh?")
    
current_index = -1
adjusted_review_stars = []
for star in review_stars:
        current_index = current_index + 1
        if current_index in skip_indexes:
            pass
        else:
            adjusted_review_stars.append(star)
            
print(len(adjusted_review_stars))
num_nodes = len(set(review_user_index_col)) + len(set(review_business_index_col))
print(num_nodes)

155429
54724


In [ ]:
num_nodes = len(set(review_user_index_col)) + len(set(review_business_index_col))

print(num_nodes)

rating_tensor = torch.tensor(adjusted_review_stars, dtype=torch.float)
print("Here 1")
edge_index = torch.tensor([review_user_index_col, review_business_index_col], dtype=torch.long)
print("Here 2")
positional_encodings = calculatePosEncodings(edge_index, num_nodes)
print("Here 3")
data = Data(edge_index=edge_index, y=rating_tensor, num_nodes=num_nodes, positional_encodings=positional_encodings)

22403
Here 1
Here 2


In [ ]:
# Step 4: Split the data into training and test sets
indices = list(range(data.edge_index.size(1)))

#das hier klein, damit der Speicher nicht überdreht wird. Aber nicht zu klein, weil sonst kommt es zu problemen!
train_indices, test_indices = train_test_split(indices, train_size=0.1, test_size=0.2)
train_indices, val_indices = train_test_split(train_indices, train_size=0.8, test_size=0.2, random_state=42)


#irgendeine syntax
train_data = data.__class__()
test_data = data.__class__()
val_data = data.__class__()


#setzt die Parameter von train_data und test_data

#soweit ich es verstehe, sind alle 2.500 nodes im training und testset vorhanden. gesplittet werden nur die edges, d.h. 
#es ist nur ein subset der 100.000 edges im training set sowie im test set vorhanden
# also 10% der Bewertungen 
train_data.edge_index = data.edge_index[:, train_indices]
train_data.y = data.y[train_indices]
train_data.num_nodes = data.num_nodes
train_data.positional_encodings = data.positional_encodings

test_data.edge_index = data.edge_index[:, test_indices]
test_data.y = data.y[test_indices]
test_data.num_nodes = data.num_nodes
test_data.positional_encodings = data.positional_encodings

val_data.edge_index = data.edge_index[:, val_indices]
val_data.y = data.y[val_indices]
val_data.num_nodes = data.num_nodes
val_data.positional_encodings = data.positional_encodings

In [ ]:
'''
Step 5: Define the Graph Convolutional Network (GCN) model. Hier könnte man dann die zusätzlichen Dinge einbauen
num_features = Anzahl input layer.  hidden_channels = Anzahl "output" layer, d.h. anzahl Parameter, die das Model versucht
sich herzuleiten und zu lernen. dadurch kann komplexität des models kontrolliert werden. wenn zB hidden_channels zu groß ist, 
dann passiert overfitting.
Soweit ich es verstehe, hat dieses Neural network nur 3 Layer: self.conv1, dann RELU, dann self.conv2. 
RELU wird hier genutzt, um non-linearity einzuführen. es ginge auch ohne. 
Actually ist der Loss ohne RELU sogar kleiner!

'''

class GCN(nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        #first graph convolution
        self.conv1 = GCNConv(num_features, hidden_channels)
        #second graph convolution
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, x, edge_index):
       # print(edge_index)
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class LSPEGCN(nn.Module):
    def __init__(self, num_features, hidden_channels, num_classes):
        super(LSPEGCN, self).__init__()
        self.gcn = GCN(num_features, hidden_channels, num_classes)


    def forward(self, x, edge_index, pos_embeddings):

        #edge features: concatenate node features and edge features (TODO when adding node features)
        x = self.gcn.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.gcn.conv2(x, edge_index)

        
        pos_embeddings_init = pos_embeddings.view(-1, pos_embeddings.size(2))
        pos_embeddings = self.gcn.conv1(pos_embeddings_init, edge_index)
        pos_embeddings = F.relu(pos_embeddings)
        pos_embeddings  = self.gcn.conv2(pos_embeddings, edge_index)
        
        return x, pos_embeddings

# Step 6: Train and evaluate the GCN model
# Set seed for reproducibility
torch.manual_seed(42)

# Set the device --> aktiviere GPU falls vorhanden
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#------------------------------------------------------

#hidden channels und epochs tunen
hidden_channels=8 #8 und 16
lr = 0.01  #0.01 vs 0.001 
epochs = 100  #100 vs 200
batch_size = 64

 #1, 16, 32 ,64, 128, 256, 512

#Early Stopping
patience = 15  # Number of epochs to wait for improvement
min_delta = 0.001  # Minimum improvement required to consider as improvement

best_val_loss = np.inf
best_epoch = 0
early_stop_counter = 0

# Define the GCNModel
#model = GATModel(num_features = 1, hidden_channels = hidden_channels, num_classes = 1, heads = 1).to(device)
#model = GINModel(num_features = 1, num_classes = 1).to(device)


model = LSPEGCN(num_features=1, hidden_channels=hidden_channels, num_classes=1)

#------------------------------------------------------
#loss function, and optimizer, MSE = Metrik für Loss 
criterion = nn.MSELoss()

optimizer = optim.Adam(model.parameters(), lr=lr)

# Create data loaders for training and test sets
train_loader = DataLoader([train_data], batch_size=batch_size)
test_loader = DataLoader([test_data], batch_size=batch_size)
val_loader = DataLoader([val_data], batch_size=batch_size)

# Model training
model.train()

train_losses = []
val_losses = []
predictions =[]

import matplotlib.pyplot as plt

for epoch in range(epochs):
    # Training
    train_loss = 0.0
    
    for batch in train_loader:
        batch = batch.to(device)
        
        out, out_pos_embeddings = model(batch.y.unsqueeze(1), batch.edge_index, batch.positional_encodings.unsqueeze(1))
        task_loss = criterion(out, batch.y)
        loss = calculateLoss(task_loss, batch, num_nodes, out_pos_embeddings)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * batch.num_graphs
        predictions = out.detach().cpu().numpy()
        #print(predictions)

    # Calculate average training loss
    train_loss /= len(train_loader.dataset)
    train_losses.append(train_loss)

    # Validation
    model.eval()
    val_loss = 0.0
    for batch in val_loader:
        batch = batch.to(device)
        
        out, out_pos_embeddings = model(batch.y.unsqueeze(1), batch.edge_index, batch.positional_encodings.unsqueeze(1))
        task_loss = criterion(out, batch.y)
        loss = calculateLoss(task_loss, batch, num_nodes, out_pos_embeddings)
        
        val_loss += loss.item() * batch.num_graphs

    # Calculate average validation loss
    val_loss /= len(val_loader.dataset)
    val_losses.append(val_loss)

    # Print training and validation loss for monitoring
    print(f"Epoch {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}")

    # Check for early stopping
    if val_loss < best_val_loss - min_delta:
        best_val_loss = val_loss
        best_epoch = epoch
        early_stop_counter = 0
    else:
        early_stop_counter += 1
        if early_stop_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Set the model back to training mode
    model.train()
'''
# Plotting training and validation curves
plt.plot(train_losses, label='Training Loss')
#plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Save the plot as an image file
plt.savefig('loss_plot.png')
'''

# Show the plot

# Evaluation on test set
model.eval()
with torch.no_grad():
    predictions = []
    targets = []

    for batch in test_loader:
        batch = batch.to(device)
        out, out_pos_embeddings = model(batch.y.unsqueeze(1), batch.edge_index, batch.positional_encodings.unsqueeze(1))
        task_loss = criterion(out, batch.y)
        test_loss = calculateLoss(task_loss, batch, num_nodes, out_pos_embeddings)
        
        print(f'Test Loss: {test_loss.item()}')
        predictions.extend(out.cpu().numpy().flatten())
        targets.extend(batch.y.cpu().numpy().flatten())

    predictions = np.array(predictions)
    targets = np.array(targets)

    rounded_predictions = np.round(predictions, decimals=0)  # Round the predicted ratings

    # Plotting the distribution
    plt.hist(rounded_predictions, bins=15, edgecolor='black')
    plt.xlabel('Predicted Rating')
    plt.ylabel('Count')
    plt.title('Distribution of Predicted Ratings')
    plt.xticks(range(1, 16))
    #plt.show()
    plt.savefig('predicted_rankings.png')


    mse = np.mean(np.abs(predictions - targets) ** 2)
    rmse = np.sqrt(mse)
    k = 5  # Define the value of k

    print(f"Batch Size: {batch_size}")
    print(f"Epochs: {epochs}")
    print   (f"MSE: {mse}")
    print(f"RMSE: {rmse}")
    
    threshold = 3.5 # Define the threshold to convert predictions into binary values
    np.set_printoptions(threshold=sys.maxsize)  # Set the threshold to print the whole array

    #print(rounded_predictions)
    #print(predictions)

    GAT_results = open("predictions_GAT.txt", "a")

    GAT_results.write(str(predictions))
    
    GAT_results.close()

    precision_value = precision(predictions, targets, threshold)
    recall_value = recall(predictions, targets, threshold)

    print(f"Precision: {precision_value}")
    print(f"Recall: {recall_value}")